# XG boosting

In [1]:
import os
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline 
import gensim
from gensim import corpora,models,similarities
from gensim.models import Word2Vec
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
df=pd.read_csv("Mock_data_combined.csv",encoding="unicode_escape")
df.head()
df1=df[["Code Chunck","Indicator"]]
df1["Indicator"]=df['Indicator'].astype(int)
df1

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Code Chunck,Indicator
0,"assert sum([1, 2, 3]) == 6,",0
1,"assert sum([1, 1, 1]) == 6,",1
2,"def test_sum():\n assert sum([1, 2, 3]) == ...",0
3,"def test_sum():\n assert sum([1, 2, 3]) == ...",1
4,def is_A_student(score):\n return score > 7...,1
5,"my_strings = ['a', 'b', 'c', 'd', 'e']\nmy_num...",1
6,"numbers = [3, 4, 6, 9, 34, 12]\n\ndef custom_s...",1
7,"def function(a):\n pass\n\nfunction(0, a=0)",1
8,def function(a):\n pass\n\nfunction(a=0),0
9,"assert sum([1, 1, 1]) == 3,",0


In [3]:
def xgboostingmodel():
    # df.columns.tolist()
    #Text Normalization
    tokenized_case = df1['Code Chunck'].apply(lambda x: x.split())
    bow_vectorizer = CountVectorizer(max_df =0.90,min_df=2,max_features=1000,stop_words='english')
    bow=bow_vectorizer.fit_transform(df1['Code Chunck'])
#     bow.shape
    tfidf_vectorizer = TfidfVectorizer(max_df =0.90,min_df=2,max_features=1000,stop_words='english')
    tfidf = tfidf_vectorizer.fit_transform(df1['Code Chunck'])
    wordvec_arrays=np.zeros((len(tokenized_case),100))
    for i in range(len(tokenized_case)):
        wordvec_arrays[i,:]=word_vector(tokenized_case[i],100)
        wordvec_df = pd.DataFrame(wordvec_arrays)
    print(wordvec_df.shape)
    return wordvec_df

In [4]:
def word_vector(tokens,size):
    tokenized_case = df1['Code Chunck'].apply(lambda x: x.split())#tokenizing
    model_w2v = gensim.models.Word2Vec(tokenized_case,size=100,window=5,min_count=2,sg=1,hs=0,negative=10,workers=2,seed=34)
    model_w2v.train(tokenized_case,total_examples = len(df1['Code Chunck']),epochs=20)
#     model_w2v.wv.most_similar(positive='assert')
#     len(model_w2v['assert'])
    vec= np.zeros(size).reshape(1,size)
    count=0.
    for word in tokens:
        try:
            vec +=model_w2v[word].reshape((1,size))
            count+=1.
        except KeyError:
            continue
    if count!=0:
        vec/=count
    return vec       

# XG boosting

In [5]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
wordvec_df=xgboostingmodel()
train_df=wordvec_df
x=train_df
target_df=df1["Indicator"]
y=target_df
y

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


(50, 100)


0     0
1     1
2     0
3     1
4     1
5     1
6     1
7     1
8     0
9     0
10    0
11    0
12    0
13    0
14    1
15    0
16    1
17    1
18    0
19    1
20    0
21    1
22    0
23    1
24    0
25    1
26    0
27    1
28    0
29    0
30    1
31    1
32    0
33    1
34    0
35    1
36    0
37    0
38    1
39    1
40    0
41    1
42    1
43    0
44    1
45    0
46    1
47    1
48    1
49    0
Name: Indicator, dtype: int32

In [7]:
# XGBoosting
# split data into train and test sets
seed = 7
test_size = 0.5
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=seed)
# print(X_test)
# print(y_test)

In [8]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
yprd_df=pd.DataFrame(predictions,index=list(X_test.index.values))
yprd_df=yprd_df.rename(columns={0:"XGB_Predicted_Indicator"})
yprd_df

[11:40:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


F:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


,XGB_Predicted_Indicator
13,1
15,1
22,1
42,0
30,1
27,0
1,0
20,1
10,1
36,1


In [9]:
# df1=df1.join(yprd_df)
# df1.to_csv("xg_prediction.csv",index=False)


final_df=df1.iloc[list(X_test.index.values),:]
final_df=pd.concat([final_df,yprd_df],axis=1)
final_df
final_df.to_csv("xg_prediction.csv",index=False)

In [10]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 32.00%


In [14]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))
print()
print("Classification Report")
print(classification_report(y_test, predictions))

Accuracy: 32.00%
Confusion Matrix:
[[ 1 11]
 [ 6  7]]

Classification Report
              precision    recall  f1-score   support

           0       0.14      0.08      0.11        12
           1       0.39      0.54      0.45        13

    accuracy                           0.32        25
   macro avg       0.27      0.31      0.28        25
weighted avg       0.27      0.32      0.29        25

